In [1]:
import pandas as pd
from matpower import start_instance

from matpowercaseframes import CaseFrames

m = start_instance()

CASE_NAME = "case9"

mpc = m.loadcase(CASE_NAME, verbose=False)
cf_mpc = CaseFrames(mpc)  # _read_oct2py_struct
cf_parse = CaseFrames(CASE_NAME)  # _read_matpower

In [15]:
for attribute in cf_mpc.attributes:
    df_mpc = getattr(cf_mpc, attribute)
    df_parse = getattr(cf_parse, attribute)

    if isinstance(df_mpc, pd.DataFrame):
        print(df_mpc.columns.equals(df_parse.columns))
        print(f"{attribute}: {df_mpc.equals(df_parse)}")
    else:
        print(f"{attribute}: {df_mpc == df_parse}")

version: True
baseMVA: True
True
bus: False
True
gen: False
True
branch: False
True
gencost: False


True